# Module 2 : Réseaux convolutionnels pour le traitement de l'image  -  Formation Cdiscount

## Mercredi 26 Mai 2021

Nicolas Baskiotis (nicolas.baskiotis@lip6.fr) Benjamin Piwowarski (benjamin.piwowarski@lip6.fr) -- MLIA/LIP6, Sorbonne Université

Les objectifs de ce module sont :
* Prise en main des réseaux convolutionnels (CNN)
* Apprentissage d'un CNN
* Introspection d'un CNN
* Fine-tuning d'un réseau pré-appris

Nous travaillerons dans un premier temps avec les données MNIST puis avec le jeu de données CIFAR d'images de 10 classes.

In [6]:
!pip install GPUtil
from GPUtil import showUtilization as gpu_usage
from numba import cuda
# !pip install torch
# !pip install torchvision
# !pip install datamaestro
from tqdm import tqdm
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import torchvision
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import time
import os
from tensorboard import notebook
# from datamaestro import prepare_dataset
from tensorboard import notebook
from torch.utils.data import TensorDataset, DataLoader,Dataset
import matplotlib.pyplot as plt

In [7]:
TB_PATH = "/tmp/logs/sceance2"
# %load_ext tensorboard
# %tensorboard --logdir /tmp/logs/sceance2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Premier réseau convolutionnel

Nous allons reprendre les données MNIST dans un premier temps.

In [8]:
TRAIN_BATCH_SIZE = 256
TEST_BATCH_SIZE = 512

# Téléchargement des données
from tensorflow.keras.datasets import mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
# mnist_ds = prepare_dataset("com.lecun.mnist")
# mnist_train_images, mnist_train_labels = mnist_ds.train.images.data(), mnist_ds.train.labels.data()
# mnist_test_images, mnist_test_labels =  mnist_ds.test.images.data(), mnist_ds.test.labels.data()

# On transforme les images en vecteurs de réels et on rescale entre 0 et 1
mnist_train_images = torch.FloatTensor(X_train).unsqueeze(1) / 255.
mnist_train_labels = torch.LongTensor(Y_train)
mnist_test_images = torch.FloatTensor(X_test).unsqueeze(1) / 255.
mnist_test_labels = torch.LongTensor(Y_test)

mnist_train_images, mnist_train_labels = mnist_train_images.to(device), mnist_train_labels.to(device)
mnist_test_images, mnist_test_labels = mnist_test_images.to(device), mnist_test_labels.to(device)

# On utilise un DataLoader pour faciliter les manipulations, on fixe arbitrairement la taille du mini batch à 32
mnist_train_loader = DataLoader(TensorDataset(mnist_train_images,mnist_train_labels),batch_size=TRAIN_BATCH_SIZE,shuffle=True)
mnist_test_loader = DataLoader(TensorDataset(mnist_test_images,mnist_test_labels),batch_size=TEST_BATCH_SIZE,shuffle=False)

On reprend la même boucle d'apprentissage.

In [9]:
def accuracy(yhat,y):
    # si y encode les indexes
    if len(y.shape)==1 or y.size(1)==1:
        return (torch.argmax(yhat,1).view(y.size(0),-1)== y.view(-1,1)).double().mean()
    # si y est encodé en onehot
    return (torch.argmax(yhat,1).view(-1) == torch.argmax(y,1).view(-1)).double().mean()


def train(model,epochs,train_loader,test_loader):
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    optim = torch.optim.Adam(model.parameters(),lr=1e-3)
    model = model.to(device)
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss()
    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x,y in train_loader:
            optim.zero_grad()
            x,y = x.to(device), y.to(device)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)
        writer.add_scalar('loss/train',cumloss/count,epoch)
        writer.add_scalar('accuracy/train',cumacc/count,epoch)
        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x,y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)
                writer.add_scalar('accuracy/test',cumacc/count,epoch)

## Réseau de convolution (CNN) 

Implémentez un réseau avec deux couches initiales de convolution <a href=https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d>**Conv2d**</a>, chacune comportant 16 filtres de taille 5x5. Chaque couche est suivie d'une activation ReLU et d'un <a href=https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html#torch.nn.MaxPool2d>**max-pooling**</a> de taille 3x3. On gardera un stride de 1 pour les convolutions et le pooling.

Quelle est la taille du tenseur de sortie des couches de convolution ? (vous pouvez consulter ce <a href=https://arxiv.org/pdf/1603.07285.pdf>guide sur l'arithmétique des convolutions</a>).

A la sortie des couches de convolutions, nous avons besoin d'un classifieur fully-connected. Utilisez deux couches de linéaires avec une activation ReLU.

Usuellement, le sous-réseau convolutionnel est stocké dans une variable *self.features* (comme son rôle est d'extraire les features de l'image), et le sous-réseau fully connected dans une variable *self.classifier*.

Implémentez la méthode **forward()** du réseau.
Entraînez votre réseau sur MNIST.

In [10]:
# Implémentation du ConvNet
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.name = "ConvNetV1"
        self.features()
        self.classifier()
        
    def features(self):
        self.conv1 = nn.Conv2d(1, 16, 5)   # black & white image -> 1 channel in ; 16 channel out ; 5 by 5 kernel ; stride = 1
        self.conv2 = nn.Conv2d(16, 256, 5) # 16 channel in ; 256 channel out ; 5 by 5 kernel ; stride = 1
        self.pool = nn.MaxPool2d(3, 3)     # 3 by 3 kernel for max pooling

    def classifier(self):
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x))) # after going through the features layers the tensor shape is : 256, 256, 1, 1
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [11]:
import warnings
warnings.filterwarnings("ignore")

# Apprentissage du ConvNet
model = ConvNet()
train(model, 1, mnist_train_loader, mnist_test_loader)

In [12]:
yhat = model(mnist_test_images)
print(yhat.shape)
print(mnist_test_labels.shape)
print(accuracy(yhat, mnist_test_labels))

## Visualisation du CNN

Une première manière d'introspecter un CNN est de visualiser les sorties des différentes couches et les filtres associés. Pour cela, on enregistre la sortie de chaque couche d'intérêt lors de la passe forward.
Le code suivant permet d'obtenir cette succession d'images : la première image est l'image originale, chaque colonne correspond à un filtre. Les résultats sont ensuite regroupés par couche de convolution, les trois images dans une même colonne correspondent :1) aux poids de la convolution, 2) la sortie de la couche de convolution, 3) la sortie du pooling.

Qu'observez vous ? Comparez les différences entre un réseau utilisant un max-pooling et un réseau utilisant un <a href="https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html">average pooling</a>. Vous pouvez également faire varier la taille et le nombre de filtres.


In [13]:
def analyse_conv(model,img,nb_filtres=16):
    print(img.shape)
    x = img.unsqueeze(0).to(device) # Modified unsqueeze because of wrong shape
    print(x.shape)
    img_conv = []
    img_pool = []
    for m in model._modules.values(): # Deleted .features
        print("Layers :", m)
        x = m.forward(x)
        if isinstance(m,nn.Conv2d):
            img_conv.append((x.squeeze(0),m.weight))
        if isinstance(m,nn.MaxPool2d) or isinstance(m,nn.AvgPool2d):
            img_pool.append(x.squeeze(0))
    plt.figure()
    plt.imshow(img.permute(1,2,0).to('cpu'),cmap='gray')
    # nombre de filtres
    ksmax = min(nb_filtres, max([p[0].size(0) for p in img_conv]))
    fig, axs = plt.subplots(3*len(img_conv),ksmax,figsize=(20,5))
    for i,((img_c,w),img_p) in enumerate(zip(img_conv,img_pool)):
        for j in range(min(nb_filtres,img_c.size(0))):
            axs[3*i,j].imshow(np.array(w[j,0].to('cpu').detach()),cmap="gray")
            axs[3*i+1,j].imshow(np.array(img_c[j].to('cpu').detach()),cmap="gray")                             
        for j in range(min(nb_filtres,img_p.size(0))):
            axs[3*i+2,j].imshow(np.array(img_p[j].to('cpu').detach()),cmap="gray")

In [14]:
# analyse_conv(model, mnist_test_images[0])

## Saliency Map

La visualisation des filtres ne permet pas bien de comprendre le rôle de chaque filtre dans la classification. Ils permettent d'extraire des features élémentaires qui combinées ensemble font sens pour un réseau fully-connected mais dont l'interprétation n'est pas évidente pour l'oeil humain.

Une première méthode pour détecter quelles zones de l'image ont le plus impacté la décision sont les cartes de saillance. L'objectif des Saliency Maps est de détecter les pixels d'entrée qui ont le plus impacté la décision. L'idée est d'utiliser le gradient *par rapport* à l'image pour ranker les pixels. En effet, un gradient fort pour un pixel d'entrée indique qu'il faut changer faiblement sa valeur pour que la classe infére change (et a contrario, un gradient nul indique que le pixel n'est pas pris en compte pour la classification selon cette classe).
Les étapes à suivre sont les suivantes :
* le flag *requires_grad* est mis à True pour l'image (pour pouvoir calculer la rétro-propagation)
* une passe forward est faite sur l'image
* le backward est calculé sur le score de sortie de la classe d'intérêt
* On affiche la valeur absolue du gradient  par rapport à l'entrée obtenu. Si l'image à plusieurs canaux, on prend le max de chacun de ces canaux.



In [15]:
def getSaliency(model,img,label):
    model.zero_grad()
    img = img.to(device)
    img.requires_grad = True
    img.grad = None
    outputs = nn.Softmax(dim=1)(model(img.unsqueeze(0)))
    output=outputs[0,label] 
    output.backward()
    sal=img.grad.abs()
    if sal.dim()>2:
        sal=torch.max(sal,dim=0)[0]
    fig=plt.figure(figsize=(8, 8))
    fig.add_subplot(1, 2, 1)
    plt.imshow(img.detach().cpu().permute(1,2,0),cmap="gray")
    fig.add_subplot(1, 2, 2)
    plt.imshow(sal.to('cpu'),cmap="seismic",interpolation="bilinear")
    return sal

In [16]:
for i in range(10):
    x,y = mnist_train_loader.dataset[i]
    # getSaliency(model,x,y)

## Données CIFAR

La base de données CIFAR10  contient  60000 images couleur (RGB) 32x32 pixels. Les images appartiennent à 10 catégories (6000 images par classe): 'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship' et 'truck'. Le
dataset est composé de 50000 exemples d'apprentissage et 10000 de test.


In [17]:
def get_stats(dataloader):
    n_batch = 0
    chan = 0
    chan_squared = 0
    for elt, _ in dataloader:
        chan += torch.mean(elt)
        chan_squared += torch.mean(elt**2)
        n_batch += 1
    mean = chan/n_batch
    std = np.sqrt((chan_squared/n_batch - mean**2))
    return mean, std

In [18]:
batchsize = 128              

cifar_trainset = torchvision.datasets.CIFAR10(root='/tmp/data', train=True, download=True, transform=transforms.ToTensor())
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batchsize, pin_memory=True, shuffle=True)

mean, std = get_stats(cifar_train_loader)

print(mean)
print(std)

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Grayscale(num_output_channels=1),
        transforms.Normalize(mean, std)
    ])

cifar_trainset = torchvision.datasets.CIFAR10(root='/tmp/data', train=True, download=True, transform=transform)
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batchsize, pin_memory=True, shuffle=True)
cifar_testset = torchvision.datasets.CIFAR10(root='/tmp/data', train=False, download=True, transform=transform)
cifar_test_loader = torch.utils.data.DataLoader(cifar_testset, batch_size=batchsize, pin_memory=True, shuffle=False)

In [19]:
print(cifar_trainset.classes)
X_train, _ = next(iter(cifar_test_loader))
print(X_train.shape)

* Testez le réseau précédent avec 32 filtres et un réseau linéaire type *Linear(in_dim,120)->ReLU->Linear(120,80)->Relu->Linear(80,10)*  sur cette base de données et comparez les résultats. 
* Expérimenter également d'autres architectures de convolution (nombre de filtres, taille des filtres, différents strides, éventuellement padding). 
* Comparez le nombre de paramètres des réseaux
* Visualisez la carte de saillance et les filtres du réseau.

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [21]:
## Définition du réseau feed-forward

class FeedFor1(nn.Module):
    def __init__(self):
        super().__init__()
        self.name = "FeedForwardV1"
        # self.fc1 = nn.Linear(32*32*3, 256)
        self.fc1 = nn.Linear(32*32, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 10)
        
    def forward(self, x):
        # x = x.view(-1,32*32*3)
        x = x.view(-1,32*32)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x))
        return x

In [22]:
## Définition du réseau convolutionnel
## Utiliser nn.init.xavier_uniform pour l'initialisation des couches de convolutions

class ConvNet2(nn.Module):
    def __init__(self):
        super().__init__()
        self.name = "ConvNetV2"
        self.features()
        self.classifier()
        
    def features(self):
        # self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv1 = nn.Conv2d(1, 32, 5)
        torch.nn.init.xavier_uniform(self.conv1.weight)
        self.conv2 = nn.Conv2d(32, 1024, 5) 
        torch.nn.init.xavier_uniform(self.conv2.weight)
        self.pool = nn.MaxPool2d(3, 3)    

    def classifier(self):
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [23]:
# feed = FeedFor1()
# conv = ConvNet2()

In [24]:
## Entraînement du réseau feed-forward
# train(feed, 10, cifar_train_loader, cifar_test_loader)

In [25]:
## Entraînement du réseau convolutionnel
# train(conv, 10, cifar_train_loader, cifar_test_loader)

In [26]:
# Affichage du nombre de paramètres
# print("Number of parameters for feedforward nn :", count_parameters(feed))
# print("Number of parameters for convolutional nn :", count_parameters(conv))

In [27]:
def get_test_data(dataloader, size):
    X_test, Y_test = next(iter(dataloader))
    batch_size = len(X_test)
    n = size//batch_size
    for i, batch in enumerate(dataloader):
        if i < n:
            X_tmp, Y_tmp = batch
            X_test = torch.cat((X_test, X_tmp), 0)
            Y_test = torch.cat((Y_test, Y_tmp), 0)
    return X_test, Y_test

X_test, Y_test = get_test_data(cifar_test_loader, len(cifar_test_loader))

X_test, Y_test = X_test.to(device), Y_test.to(device)

print(X_test.shape)
print(Y_test.shape)

In [28]:
# yha_feed = feed(X_test)
# print(yhat.shape)
# print("Acc for feedforward nn : ", accuracy(yha_feed, Y_test))

# yhat_conv = conv(X_test)
# print(yhat.shape)
# print("Acc for convolutional nn :", accuracy(yhat_conv, Y_test))

In [29]:
## Analyse des filtres du réseau
# analyse_conv(conv, )

In [30]:
## Carte de saillance du réseau
for i in range(10):
    x,y = cifar_train_loader.dataset[i]
    # getSaliency(conv,x,y)

# Data Augmentation

Pour améliorer les résultats, une technique courante est d'augmenter les données par des variantes des images du corpus. Cela permet de gagner en robustesse vis à vis de diverses transformations en forçant le réseau à apprendre des invariants (e.g. d'échelle, de rotation, d'inversion, de luminosité, etc.). 

Insérez quelques transformations de données lors du chargement des données (la liste des transformations disponibles se trouvent dans <a href=https://pytorch.org/vision/stable/transforms.html> torchvision.transforms</a>, par exemple **RandomHorizontalFlip()**, **RandomResizedCrop()**) et relancez l'apprentissage pour voir l'effet. Les transformations sont à insérer dans le **transforms.Compose()** avant la transformation en tenseur.

In [31]:
## Définition de la transformation pour Data Augmentation et création du réseau et des dataloader.
batchsize = 64            

cifar_trainset = torchvision.datasets.CIFAR10(root='/tmp/data', train=True, download=True, transform=transforms.ToTensor())
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batchsize, pin_memory=True, shuffle=True)

mean, std = get_stats(cifar_train_loader)

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Grayscale(num_output_channels=1),
        transforms.Normalize(mean, std),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomResizedCrop(32*32)
    ])

cifar_trainset = torchvision.datasets.CIFAR10(root='/tmp/data', train=True, download=True, transform=transform)
cifar_train_loader = torch.utils.data.DataLoader(cifar_trainset, batch_size=batchsize, pin_memory=True, shuffle=True)
cifar_testset = torchvision.datasets.CIFAR10(root='/tmp/data', train=False, download=True, transform=transform)
cifar_test_loader = torch.utils.data.DataLoader(cifar_testset, batch_size=batchsize, pin_memory=True, shuffle=False)

In [32]:
# import gc

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()     
#     gc.collect()
#     torch.cuda.empty_cache()
#     cuda.select_device(0)
#     cuda.close()
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     cuda.select_device(0)
#     print("GPU Usage after emptying the cache")
#     gpu_usage()
#     return device

# if feed : del feed
# if conv : del conv
# device = free_gpu_cache() 

In [33]:
conv = ConvNet2()

In [34]:
## Apprentissage du réseau
# train(conv, 10, cifar_train_loader, cifar_test_loader)

In [35]:
## Accuracy
# X_test, Y_test = get_test_data(cifar_test_loader)
# X_test, Y_test = X_test.to(device), Y_test.to(device)
# print("Acc for convolutional nn :", accuracy(conv(X_test), Y_test))

# Modèles pré-entraînés / Transfert

PyTorch propose un certain nombre de modèles pré-entraînés sur le très gros corpus d'images ImageNet. Ces modèles très lourds demandent beaucoup de ressources pour être entraînés efficacement. Mais une fois leur entraînement effectué, ils peuvent être appliqués assez facilement sur d'autres corpus que ImageNet, moyennant quelques adaptations. Dans la suite nous considérons le modèle <a href=https://pytorch.org/hub/pytorch_vision_alexnet/>AlexNet</a> pour l'extraction de features. La sortie du réseau doit être adaptée et ré-entraînée pour permettre de classer des images sur notre corpus CIFAR. 

Commençons par collecter le réseau entraîné et étudions sa structure: 

In [36]:
from torchvision import models
alexnet = models.alexnet(pretrained=True)
print(alexnet)

## Fine-Tuning d'AlexNet

Que faut-il modifier pour l'adapter à notre cas ? En outre on aimerait que lors de l'apprentissage seuls les poids des modules modifiés soient ajustés. Penser à fixer les autres.

In [37]:
# Il faut modifier le nombre de neurones en sortie car nous n'avons que 10 classes à prédire.
# Il faut modifier le nombre de neurones sur les couches dans le classifieur car il risque fortement d'overfit, du fait de notre nombre de classes.

alexnet.classifier[1] = nn.Linear(9216, 2048)
alexnet.classifier[4] = nn.Linear(2048, 1024)
alexnet.classifier[6] = nn.Linear(1024, 10)
alexnet.eval()

In [38]:
# Ici nous voulons entrainer seulement la partie classifieur et non la partie feature extracting du CNN
# Permet de mettre à True/False tous les requires_grad des paramètres du réseau

def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for name,p in model.named_parameters():
            if "features" in name:
                p.requires_grad = False    
            else:
                p.requires_grad = True    
            
set_parameter_requires_grad(alexnet, True)

Il s'agit également de remettre le modèle dans les mêmes conditions qu'il a été appris (taille de l'entrée 224, normalisation selon moyennes et variances de ImageNet, etc.).

In [39]:
input_size=224

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

transformAlexTrain=transforms.Compose([ # Cette fois on utilise pas de grayscale car nous avons un gros modele pré-entrainé
        transforms.RandomResizedCrop(input_size), # selection aléatoire d'une zone de la taille voulue (augmentation des données en apprentissage)
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
transformAlexTest=transforms.Compose([
        transforms.Resize(input_size), # selection de la zone centrale de la taille voulue
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

alex_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transformAlexTrain)
alex_trainloader = torch.utils.data.DataLoader(alex_trainset, batch_size=batchsize, pin_memory=True, shuffle=True)

alex_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transformAlexTest)
alex_testloader = torch.utils.data.DataLoader(alex_testset, batch_size=batchsize, pin_memory=True, shuffle=True)

In [40]:
def train(model,epochs,train_loader,test_loader,feature_extract=False):
    model = model.to(device)
    writer = SummaryWriter(f"{TB_PATH}/{model.name}")
    
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print(name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print(name)
    optim = torch.optim.Adam(params_to_update,lr=1e-3)
    
    print(f"running {model.name}")
    loss = nn.CrossEntropyLoss()
    for epoch in tqdm(range(epochs)):
        cumloss, cumacc, count = 0, 0, 0
        model.train()
        for x,y in train_loader:
            optim.zero_grad()
            x,y = x.to(device), y.to(device)
            yhat = model(x)
            l = loss(yhat,y)
            l.backward()
            optim.step()
            cumloss += l*len(x)
            cumacc += accuracy(yhat,y)*len(x)
            count += len(x)
        writer.add_scalar('loss/train',cumloss/count,epoch)
        writer.add_scalar('accuracy/train',cumacc/count,epoch)
        if epoch % 1 == 0:
            model.eval()
            with torch.no_grad():
                cumloss, cumacc, count = 0, 0, 0
                for x,y in test_loader:
                    x,y = x.to(device), y.to(device)
                    yhat = model(x)
                    cumloss += loss(yhat,y)*len(x)
                    cumacc += accuracy(yhat,y)*len(x)
                    count += len(x)
                writer.add_scalar(f'loss/test',cumloss/count,epoch)
                writer.add_scalar('accuracy/test',cumacc/count,epoch)

Faites le Fine-tuning de alexnet sur les données CIFAR. Regardez les cartes de saillances obtenues.

In [41]:
## Entraînement du réseau
alexnet.name = "AlexNet"
# train(alexnet, 8, alex_trainloader, alex_testloader)

In [42]:
## Accuracy
# X_test, Y_test = get_test_data(alex_testloader, 3000)
# X_test, Y_test = X_test.to(device), Y_test.to(device)
# print("Acc for alexnet transfer learning :", accuracy(alexnet(X_test), Y_test))

In [43]:
## Carte de saillance du réseau
# inputs,labels=iter(alex_testloader).next()
# for i in range(len(cifar_trainset.classes)):
#     print("Pour ",cifar_trainset.classes[i])
#     getSaliency(alexnet,inputs[0],i)

## Class Activation Maps (CAM)
Une autre technique d'introspection est le Class Activation Maps. Cette technique permet de visualiser quels sont les régions qui ont fait le plus réagir les différents filtres qui ont servis à la classification. Elle part de la constatation que la sortie d'un filtre de la dernière couche convolutionnelle indique spatialement quelles sont les régions de l'image qui ont fait réagir le filtre (la sortie est généralement de taille plus petite - *downscalé* - mais on peut la mettre à l'échelle). Cependant, il est difficile d'analyser avec la succession des couches non-linéaires en aval le rôle de chaque sortie convolutionnelle dans le processus de classification. Cependant, un réseau plus simple - uniquement linéaire par exemple - permettrait de donner une indication à l'importance de chaque filtre (au prix d'une erreur plus grosse en classification). 
Cette technique nécessite donc la modification des dernières couches du réseau de la manière suivante : 
* un pooling de moyennage globale (un average pooling de la taille de l'image) est appliquée à chaque filtre de convolution de la dernière couche convolutionnelle : seul le signal moyen de chaque filtre est retenu, sans plus aucune information spatiale.
* un réseau linéaire est ensuite utilisé du nombre de filtres vers le nombre de classes qui va permettre de mettre en évidence l'intérêt de chaque filtre dans la classification.

Le réseau ainsi modifié est fine-tuné sur le corpus. L'Activation Map est obtenu en sommant les sorties de la dernière couche convolutionnelle pondérées par les poids du réseau linéaire. 
Modifiez le réseau, puis ré-entraîneé les couches modifiées. 

In [64]:
# Couche identité
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x

In [83]:
## Remplacement de la couche classifier par un module d'average pooling et un linéaire.
alexnet = models.alexnet(pretrained=True)
print(alexnet)
alexnet.features[12] = nn.AvgPool2d(kernel_size=3, stride=2, padding=0, ceil_mode=False)
alexnet.classifier[0] = Identity()
alexnet.classifier[1] = Identity()
alexnet.classifier[2] = Identity()
alexnet.classifier[3] = Identity()
alexnet.classifier[4] = Identity()
alexnet.classifier[5] = Identity()
alexnet.classifier[6] = nn.Linear(9216, 10)
print(alexnet)

In [84]:
## Entrainement du réseau
set_parameter_requires_grad(alexnet, True)
alexnet.name = "AlexNetAvg"
train(alexnet, 1, alex_trainloader, alex_testloader)

Il ne reste plus qu'à écrire la fonction generate_cam qui affiche une image d'activation par classe.

In [85]:
def generate_cam(model,input_image,target_class=None):
    ## Calcul du forward sur l'image
    with torch.no_grad():
        input_image=input_image.to(device)
        x = model.features(input_image)
        out=model.classifier(x)
        out=torch.nn.functional.softmax(out,-1)
    if target_class is None:
        target_class = torch.max(out,dim=-1)[1].item()
    print("target_class",target_class)
    ## Récupération des poids du linéaire
    weights = dict(model.classifier.named_modules())["Linear"].weight.data  
    fig = plt.figure(figsize=(16, 8))
    fig.add_subplot(1,2, 1)
    img=input_image.to("cpu")*torch.tensor(std).view(3,1,1)+torch.tensor(mean).view(3,1,1)
    img=torch.nn.functional.interpolate(img, size=(244, 244), mode="bilinear", align_corners=False)
    plt.imshow(img.cpu().squeeze().permute(1,2,0))
    ## Calcul de CAM
    y=x*weights[target_class].view(1,-1,1,1)
    y=(y.sum(1))  
    fig.add_subplot(1, 2, 2)
    y=torch.nn.functional.interpolate(y.unsqueeze(0),size=(244,244),mode="bilinear",align_corners=False)
    plt.imshow(y.cpu().squeeze(),cmap="afmhot")
    plt.show()

In [86]:
# inputs,labels=iter(alex_trainloader).next()
# generate_cam(alexnet,inputs[0].unsqueeze(0))

On peut aussi charger des images du web et voir ce que notre classifieur donne. Par exemple:

In [87]:
# !wget "https://www.fidanimo.com/sites/default/files/2020-10/dog-sitter.jpg"
# !wget https://assets.siemens-energy.com/siemens/assets/api/uuid:78a9c83e-219e-4fd5-b948-2bf07276916d/width:640/quality:high/4320x3240-keyvisual-cargo.jpg
# from PIL import Image
# imageDog = transformAlexTest(Image.open("dog-sitter.jpg")).unsqueeze(0).to(device, torch.float)
# imageShip = transformAlexTest(Image.open("4320x3240-keyvisual-cargo.jpg")).unsqueeze(0).to(device,torch.float)

In [88]:
# for i in range(10):
#     print(cifar_trainset.classes[i])
#     generate_cam(alexnet,imageDog,i)
# generate_cam(alexnet,imageShip)